<span style="color:red"> **목표1: 재발 유무에 따라 데이터 합치기**
    
[작업순서]
- 테이블별 필요변수 추출 (자격 및 보험료, 진료테이블(T300, T530), 암검진 제외
- 재발시기 DAY 값 추출(T200-RECU_TO_DD에서 U99시점의 RECU_FR_DD)
- 재발 유무에 따라 데이터 merge하기

# 필요 라이브러리

In [1]:
import os
import pandas as pd

----

# 1. 경로설정
- 현재 경로 파악하기 위함
- 데이터를 바로 불러올 수 있는 경로 설정 완료

In [2]:
print(os.getcwd()) # 현재 경로
# print(os.listdir(os.getcwd())) # 현재 경로 파일

os.chdir('/project/kcure-08/src/1. Data/1. Colon') # 현재 경로 변경(데이터 파일)
print(os.getcwd())
# print(os.listdir(os.getcwd()))

/project/kcure-08/src/2. Model/1. Colon
/project/kcure-08/src/1. Data/1. Colon


----

# 2. 데이터 불러오기

In [3]:
df_cc_rgst = pd.read_csv('./Raw_csv/cs_cc_rgst.csv') # 병기조사 및 사망원인
df_cc_bfc = pd.read_csv('./Raw_csv/cs_cc_bfc.csv') # 자격 및 보험료
df_cc_t200 = pd.read_csv('./Raw_csv/cs_cc_t200.csv') # 진료(명세서)
df_cc_t300 = pd.read_csv('./Raw_csv/cs_cc_t300.csv') # 진료(진료내역)
df_cc_t400 = pd.read_csv('./Raw_csv/cs_cc_t400.csv') # 진료(상병내역)
df_cc_t530 = pd.read_csv('./Raw_csv/cs_cc_t530.csv',) # 진료(원외처방내역)
df_cc_g1e_obj = pd.read_csv('./Raw_csv/cs_cc_g1e_obj.csv') # 건강검진 (대상자)
df_cc_g1eq_1217 = pd.read_csv('./Raw_csv/cs_cc_g1eq_1217.csv') # 건강검진 (문진, 결과 및 판정)
df_cc_gq_1821 = pd.read_csv('./Raw_csv/cs_cc_gq_1821.csv') # 건강검진 (문진)
df_cc_g1e_1821 = pd.read_csv('./Raw_csv/cs_cc_g1e_1821.csv') # 건강검진 (결과 및 판정)
df_cc_ccr_cq_rst_1217 = pd.read_csv('./Raw_csv/cs_cc_ccr_cq_rst_1217.csv') # 암문진()
df_cc_ccr_cq_rst_1821 = pd.read_csv('./Raw_csv/cs_cc_ccr_cq_rst_1821.csv') # 암문진 ()
df_cc_ccr_rst_1217 = pd.read_csv('./Raw_csv/cs_cc_ccr_rst_1217.csv') # 암검진()
df_cc_ccr_rst_1821 = pd.read_csv('./Raw_csv/cs_cc_ccr_rst_1821.csv') # 암검진()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (12,13,14,15,16,30,31,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython

----

# 3. 데이터 전처리

In [20]:
### object형 데이터 한번에 처리
def process_column(value):
    value = value[1:]
    value = value.replace("'", "", 2)
    return value

In [31]:
### object형 데이터 한번에 처리
def process_column(value):
    if isinstance(value, str):
        value = value[1:]
        value = value.replace("'", "", 2)
    return value

### 1. 병기조사 및 사망원인 테이블 (;rgst)

- Histo-patholgy report : grade(분화도), 

In [28]:
df_cc_rgst = df_cc_rgst.sort_values(by = 'SN_KEY', ascending = True)

# 불필요한 변수 제거 #
df_cc_rgst = df_cc_rgst.drop(columns=['AJCC7_T', 'AJCC7_TEVAL', 'AJCC7_N', 'AJCC7_NEVAL', 'AJCC7_M', 'AJCC7_MEVAL', 'AJCC7_STAGE', 'SAIN1', 'SAIN2'])

In [32]:
o_col = df_cc_rgst.select_dtypes(include='object').columns
df_cc_rgst[o_col] = df_cc_rgst[o_col].applymap(process_column) # 'b', '' 제거

In [63]:
### csv 파일 저장 ###
df_cc_rgst.to_csv('./PP_csv/cs_cc_rgst.csv', index = False) 

------

### 2-1. 진료 테이블(명세서 일반내역; T200)
- Col = ['SN_KEY', 'MID', 'M_SICK_CD', 'SUB_SICK_CD', 'CL_CD','RVD_PL_CD','RECU_FR_DD','RECU_TO_DD','SOPR_YN','DGRSLT_TP_CD']

In [34]:
df_cc_t200 = df_cc_t200.sort_values(by = 'SN_KEY', ascending = True)

### 불필요한 변수 제거 ###
df_cc_t200 = df_cc_t200[['SN_KEY', 'MID', 'M_SICK_CD', 'SUB_SICK_CD', 'CL_CD','RVD_PL_CD','RECU_FR_DD','RECU_TO_DD','SOPR_YN','DGRSLT_TP_CD']]

In [35]:
o_col = df_cc_t200.select_dtypes(include='object').columns
df_cc_t200[o_col] = df_cc_t200[o_col].applymap(process_column) # 'b', '' 제거

In [ ]:
# datetime으로 변환
df_cc_t200['RECU_FR_DD'] = pd.to_datetime(df_cc_t200['RECU_FR_DD'])
df_cc_t200['RECU_TO_DD'] =  pd.to_datetime(df_cc_t200['RECU_TO_DD'])

In [56]:
### csv 파일 저장 ###
df_cc_t200.to_csv('./PP_csv/cs_cc_t200.csv', index = False)

-----

### 2-2. 진료 테이블(상병내역; T400)
- Col = [SN_KEY, MID,	SICK_CD_NEW, SICK_TY_CD]

In [47]:
df_cc_t400 = df_cc_t400.sort_values(by = 'SN_KEY', ascending = True)

In [52]:
df_cc_t400['SICK_TY_CD'] = df_cc_t400['SICK_TY_CD'].str[1:]
df_cc_t400['SICK_TY_CD'] = df_cc_t400['SICK_TY_CD'].str.replace("'", "", 2)

In [ ]:
o_col = df_cc_t400.select_dtypes(include='object').columns
df_cc_t400[o_col] = df_cc_t400[o_col].applymap(process_column) # 'b', '' 제거

In [54]:
### csv 파일 저장 ###
df_cc_t400.to_csv('./PP_csv/cs_cc_t400.csv', index = False)

----

### 3. 건강검진 테이블

In [3]:
df_cc_g1e_obj = pd.read_csv('./Raw_csv/cs_cc_g1e_obj.csv') # 건강검진 (대상자)
df_cc_g1eq_1217 = pd.read_csv('./Raw_csv/cs_cc_g1eq_1217.csv') # 2012 - 2017 건강검진 (문진, 결과 및 판정)
df_cc_gq_1821 = pd.read_csv('./Raw_csv/cs_cc_gq_1821.csv') # 2018 - 2021 건강검진 (문진)
df_cc_g1e_1821 = pd.read_csv('./Raw_csv/cs_cc_g1e_1821.csv') # 2018 - 2021 건강검진 (결과 및 판정)

In [14]:
### 오름차순 정렬 ###
df_cc_g1e_obj = df_cc_g1e_obj.sort_values(by = ['SN_KEY','EXMD_BZ_YYYY'], ascending = True) # 건강검진 (대상자)
df_cc_g1eq_1217 = df_cc_g1eq_1217.sort_values(by = ['SN_KEY','EXMD_BZ_YYYY'], ascending = True) # 건강검진 (문진, 결과 및 판정)
df_cc_gq_1821 = df_cc_gq_1821.sort_values(by = ['SN_KEY','EXMD_BZ_YYYY'], ascending = True) # 건강검진 (문진)
df_cc_g1e_1821 = df_cc_g1e_1821.sort_values(by = ['SN_KEY','EXMD_BZ_YYYY'], ascending = True) # 건강검진 (결과 및 판정)

In [15]:
### 불필요한 변수 제거 ###
df_cc_g1eq_1217 = df_cc_g1eq_1217.drop(columns=['Q_FHX_HTN', 'Q_FHX_STK', 'Q_FHX_HTDZ', 'Q_FHX_DM'])
df_cc_gq_1821 = df_cc_gq_1821.drop(columns=['Q_FHX_HTN', 'Q_FHX_STK', 'Q_FHX_HTDZ', 'Q_FHX_DM'])

In [ ]:
o_col = df_cc_g1eq_1217.select_dtypes(include='object').columns
df_cc_g1eq_1217[o_col] = df_cc_g1eq_1217[o_col].applymap(process_column) # 'b', '' 제거

In [ ]:
o_col = df_cc_gq_1821.select_dtypes(include='object').columns
df_cc_gq_1821[o_col] = df_cc_gq_1821[o_col].applymap(process_column) # 'b', '' 제거

In [17]:
### 건강검진 데이터 합치기 전 변수명 변경 ###

df_cc_g1eq_1217.columns = (['SN_KEY', 'EXMD_BZ_YYYY', 'G1E_HGHT_1217', 'G1E_WGHT_1217', 'G1E_BMI_1217', 'G1E_WSTC_1217',
       'G1E_BP_SYS_1217', 'G1E_BP_DIA_1217', 'G1E_URN_PROT_1217', 'G1E_HGB_1217', 'G1E_FBS_1217',
       'G1E_TOT_CHOL_1217', 'G1E_SGOT_1217', 'G1E_SGPT_1217', 'G1E_GGT_1217', 'G1E_TG_1217', 'G1E_HDL_1217',
       'G1E_LDL_1217', 'G1E_CRTN_1217', 'G1E_GFR_1217', 'Q_FHX_ETC_1217', 'G1E_PHX_YN_1217', 'Q_PHX_DX_STK_1217',
       'Q_PHX_DX_HTDZ_1217', 'Q_PHX_DX_HTN_1217', 'Q_PHX_DX_DM_1217', 'Q_PHX_DX_PTB_1217',
       'Q_PHX_DX_DLD_1217', 'Q_PHX_DX_ETC_1217', 'Q_PHX_TX_STK_1217', 'Q_PHX_TX_HTDZ_1217',
       'Q_PHX_TX_HTN_1217', 'Q_PHX_TX_DM_1217', 'Q_PHX_TX_DLD_1217', 'Q_PHX_TX_ETC_1217',
       'Q_SMK_YN_1217', 'Q_DRK_FRQ_V09N_1217', 'Q_DRK_AMT_V09N_1217', 'Q_PA_VD_1217', 'Q_PA_MD_1217',
       'Q_PA_WALK_1217'])


df_cc_gq_1821.columns = (['SN_KEY', 'EXMD_BZ_YYYY', 'Q_PHX_DX_STK_1821', 'Q_PHX_DX_HTDZ_1821',
       'Q_PHX_DX_HTN_1821', 'Q_PHX_DX_DM_1821', 'Q_PHX_DX_DLD_1821', 'Q_PHX_DX_PTB_1821',
       'Q_PHX_DX_ETC_1821', 'Q_PHX_TX_STK_1821', 'Q_PHX_TX_HTDZ_1821', 'Q_PHX_TX_HTN_1821',
       'Q_PHX_TX_DM_1821', 'Q_PHX_TX_DLD_1821', 'Q_PHX_TX_PTB_1821', 'Q_PHX_TX_ETC_1821',
       'Q_FHX_ETC_1821', 'Q_HBV_AG_1821', 'Q_DRK_PER_1821', 'Q_DRK_FRQ_1821', 'Q_DRK_SOJU_SHOT_1821',
       'Q_DRK_SOJU_BTL_1821', 'Q_DRK_SOJU_CAN_1821', 'Q_DRK_SOJU_CC_1821', 'Q_DRK_BEER_SHOT_1821',
       'Q_DRK_BEER_BTL_1821', 'Q_DRK_BEER_CAN_1821', 'Q_DRK_BEER_CC_1821', 'Q_DRK_LQR_SHOT_1821',
       'Q_DRK_LQR_BTL_1821', 'Q_DRK_LQR_CAN_1821', 'Q_DRK_LQR_CC_1821', 'Q_DRK_KLQR_SHOT_1821',
       'Q_DRK_KLQR_BTL_1821', 'Q_DRK_KLQR_CAN_1821', 'Q_DRK_KLQR_CC_1821', 'Q_DRK_WINE_SHOT_1821',
       'Q_DRK_WINE_BTL_1821', 'Q_DRK_WINE_CAN_1821', 'Q_DRK_WINE_CC_1821',
       'Q_DRK_MAX_SOJU_SHOT_1821', 'Q_DRK_MAX_SOJU_BTL_1821', 'Q_DRK_MAX_SOJU_CAN_1821',
       'Q_DRK_MAX_SOJU_CC_1821', 'Q_DRK_MAX_BEER_SHOT_1821', 'Q_DRK_MAX_BEER_BTL_1821',
       'Q_DRK_MAX_BEER_CAN_1821', 'Q_DRK_MAX_BEER_CC_1821', 'Q_DRK_MAX_LQR_SHOT_1821',
       'Q_DRK_MAX_LQR_BTL_1821', 'Q_DRK_MAX_LQR_CAN_1821', 'Q_DRK_MAX_LQR_CC_1821',
       'Q_DRK_MAX_KLQR_SHOT_1821', 'Q_DRK_MAX_KLQR_BTL_1821', 'Q_DRK_MAX_KLQR_CAN_1821',
       'Q_DRK_MAX_KLQR_CC_1821', 'Q_DRK_MAX_WINE_SHOT_1821', 'Q_DRK_MAX_WINE_BTL_1821',
       'Q_DRK_MAX_WINE_CAN_1821', 'Q_DRK_MAX_WINE_CC_1821', 'Q_SMK_YN_1821', 'Q_SMK_DRT_1821',
       'Q_SMK_NOW_YN_1821', 'Q_SMK_PRE_DRT_1821', 'Q_SMK_PRE_AMT_1821', 'Q_SMK_AMT_1821',
       'Q_SMK_NOW_DRT_1821', 'Q_SMK_NOW_AMT_V09N_1821', 'Q_SMK_STP_DRT_1821', 'Q_HNBT_YN_1821',
       'Q_HNBT_NOW_YN_1821', 'Q_HNBT_DRT_1821', 'Q_HNBT_AMT_1821', 'Q_HNBT_STP_DRT_1821',
       'Q_E_CIGA_YN_1821', 'Q_E_CIGA_AMT_1821', 'Q_PA_VD_FRQ_1821', 'Q_PA_VD_HRS_1821',
       'Q_PA_VD_MINS_1821', 'Q_PA_MD_FRQ_1821', 'Q_PA_MD_HRS_1821', 'Q_PA_MD_MINS_1821',
       'Q_PA_MUSL_FRQ_1821'])

df_cc_g1e_1821.columns = (['SN_KEY', 'EXMD_BZ_YYYY', 'G1E_HGHT_1821', 'G1E_WGHT_1821', 'G1E_BMI_1821', 'G1E_WSTC_1821',
       'G1E_BP_SYS_1821', 'G1E_BP_DIA_1821', 'G1E_URN_PROT_1821', 'G1E_HGB_1821', 'G1E_FBS_1821',
       'G1E_TOT_CHOL_1821', 'G1E_TG_1821', 'G1E_HDL_1821', 'G1E_LDL_1821', 'G1E_CRTN_1821', 'G1E_SGOT_1821',
       'G1E_SGPT_1821', 'G1E_GGT_1821', 'G1E_GFR_1821', 'G1E_PHX_YN_1821', 'G1E_PHX_DRUG_YN_1821'])

In [ ]:
df_cc_g1e_obj.drop_duplicates('SN_KEY')
df_cc_g1eq_1217.drop_duplicates('SN_KEY')
df_cc_gq_1821.drop_duplicates('SN_KEY')

In [26]:
### 합치기 ###
df_cc_g1e_1821_tmp = pd.merge(df_cc_gq_1821, df_cc_g1e_1821, how = 'inner', on = ['SN_KEY', 'EXMD_BZ_YYYY'])
df_cc_g1e_1821 = pd.merge(df_cc_g1e_obj, df_cc_g1e_1821_tmp, how = 'inner', on = ['SN_KEY', 'EXMD_BZ_YYYY']) # 일반건강검진(2018-2021)
df_cc_g1e_1217 = pd.merge(df_cc_g1e_obj, df_cc_g1eq_1217, how = 'inner', on = ['SN_KEY', 'EXMD_BZ_YYYY']) # 일반건강검진(2012-2017)
df_cc_g1e = pd.merge(df_cc_g1e_1217, df_cc_g1e_1821, how = 'outer', on = ['SN_KEY', 'EXMD_BZ_YYYY']) # 일반건강검진(2012-2017)

In [ ]:
df_cc_g1e_1821_tmp = pd.merge(df_cc_gq_1821, df_cc_g1e_1821, how = 'inner', on = ['SN_KEY', 'EXMD_BZ_YYYY'])
df_cc_g1e_1821_tmp.drop_duplicates('SN_KEY') # 9927명

df_cc_g1e_1821 = pd.merge(df_cc_g1e_obj, df_cc_g1e_1821_tmp, how = 'inner', on = ['SN_KEY', 'EXMD_BZ_YYYY'])
df_cc_g1e_1821.drop_duplicates('SN_KEY') # 9786명

df_cc_g1e_1217 = pd.merge(df_cc_g1e_obj, df_cc_g1eq_1217, how = 'inner', on = ['SN_KEY', 'EXMD_BZ_YYYY']) # 일반건강검진(2012-2017)
df_cc_g1e_1217.drop_duplicates('SN_KEY') # 15446명

In [30]:
### csv 파일 저장 ###
df_cc_g1e.to_csv('./PP_csv/cs_cc_g1e.csv', index = False) 

-----

### 4. 암검진 테이블

In [44]:
df_cc_ccr_cq_rst_1217 = pd.read_csv('./Raw_csv/cs_cc_ccr_cq_rst_1217.csv') # 암문진()
df_cc_ccr_cq_rst_1821 = pd.read_csv('./Raw_csv/cs_cc_ccr_cq_rst_1821.csv') # 암문진 ()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (12,13,14,15,16,30,31,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
### 오름차순 정렬 ###
df_cc_ccr_cq_rst_1217 = df_cc_ccr_cq_rst_1217.sort_values(by = ['SN_KEY','EXMD_BZ_YYYY'], ascending = True) # 건강검진 (대상자)
df_cc_ccr_cq_rst_1821 = df_cc_ccr_cq_rst_1821.sort_values(by = ['SN_KEY','EXMD_BZ_YYYY'], ascending = True) # 건강검진 (문진, 결과 및 판정)

In [47]:
df_cc_ccr_cq_rst_1217 = df_cc_ccr_cq_rst_1217[['SN_KEY', 'EXMD_BZ_YYYY', 'QC_SX_YN', 'QC_WT_LSS', 'QC_WT_LSS_AMT',
       'QC_PFHX_CST_YN', 'QC_PFHX_CST_I', 'QC_PFHX_CBR_YN', 'QC_PFHX_CBR_I', 'QC_PFHX_CCR_YN', 'QC_PFHX_CCR_I', 'QC_PFHX_CLV_YN', 'QC_PFHX_CLV_I', 'QC_PFHX_CCX_YN', 'QC_PFHX_CCX_I','QC_PFHX_ETC_YN', 'QC_PFHX_ETC_I', 'QC_TST_UGI_YN', 'QC_TST_GSC_YN', 'QC_TST_MMG_YN', 'QC_TST_FOBT_YN',
       'QC_TST_DCBE_YN', 'QC_TST_CSC_YN', 'QC_TST_PPSM_YN', 'QC_TST_LVUS_YN',
       'QC_PHX_PU_YN', 'QC_PHX_ATGS_YN', 'QC_PHX_INMT_YN', 'QC_PHX_GPO_YN',
       'QC_PHX_ST_ETC', 'QC_PHX_CRA_YN', 'QC_PHX_CD_YN', 'QC_PHX_HMR_YN',
       'QC_PHX_LV_YN', 'QC_PHX_CHB_YN', 'QC_PHX_CHC_YN']]

df_cc_ccr_cq_rst_1821 = df_cc_ccr_cq_rst_1821[['SN_KEY', 'EXMD_BZ_YYYY', 'QC_SX_YN', 'QC_WT_LSS', 'QC_WT_LSS_AMT',
       'QC_PFHX_CST_YN', 'QC_PFHX_CST_I', 'QC_PFHX_CBR_YN', 'QC_PFHX_CBR_I', 'QC_PFHX_CCR_YN', 'QC_PFHX_CCR_I', 'QC_PFHX_CLV_YN', 'QC_PFHX_CLV_I', 'QC_PFHX_CCX_YN', 'QC_PFHX_CCX_I','QC_PFHX_ETC_YN', 'QC_PFHX_ETC_I', 'QC_TST_UGI_YN', 'QC_TST_GSC_YN', 'QC_TST_MMG_YN', 'QC_TST_FOBT_YN',
       'QC_TST_DCBE_YN', 'QC_TST_CSC_YN', 'QC_TST_PPSM_YN', 'QC_TST_LVUS_YN',
       'QC_PHX_PU_YN', 'QC_PHX_ATGS_YN', 'QC_PHX_INMT_YN', 'QC_PHX_GPO_YN',
       'QC_PHX_ST_ETC', 'QC_PHX_CRA_YN', 'QC_PHX_CD_YN', 'QC_PHX_HMR_YN',
       'QC_PHX_LV_YN', 'QC_PHX_CHB_YN', 'QC_PHX_CHC_YN']]

In [ ]:
o_col = df_cc_ccr_cq_rst_1217.select_dtypes(include='object').columns
df_cc_ccr_cq_rst_1217[o_col] = df_cc_ccr_cq_rst_1217[o_col].applymap(process_column) # 'b', '' 제거

In [ ]:
o_col = df_cc_ccr_cq_rst_1821.select_dtypes(include='object').columns
df_cc_ccr_cq_rst_1821[o_col] = df_cc_ccr_cq_rst_1821[o_col].applymap(process_column) # 'b', '' 제거

In [58]:
### 합치기 ###
df_cc_ccr_cq_rst = pd.merge(df_cc_ccr_cq_rst_1217, df_cc_ccr_cq_rst_1821, how = 'outer', on = ['SN_KEY', 'EXMD_BZ_YYYY']) # 일반건강검진(2012-2017)
df_cc_ccr_cq_rst = df_cc_ccr_cq_rst.sort_values(by = ['SN_KEY','EXMD_BZ_YYYY'], ascending = True) # 건강검진 (대상자)

In [60]:
### csv 파일 저장 ###
df_cc_ccr_cq_rst.to_csv('./PP_csv/cs_cc_ccr_cq_rst.csv', index = False) 

---

# 4. 데이터 합치기

In [31]:
df_cc_t200 = pd.read_csv('./PP_csv/cs_cc_t200.csv') # 8972800 rows × 10 columns
df_cc_t400 = pd.read_csv('./PP_csv/cs_cc_t400.csv') # 21897420 rows × 4 columns

df_cc_rgst = pd.read_csv('./PP_csv/cs_cc_rgst.csv') # 52015 rows × 144 columns
df_cc_g1e = pd.read_csv('./PP_csv/cs_cc_g1e.csv') # 21773 rows × 36 columns
df_cc_ccr_cq_rst = pd.read_csv('./PP_csv/cs_cc_ccr_cq_rst.csv') # 32598 rows × 70 columns

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
### 테이블 합치기 ###
df_cs_cc = pd.merge(df_cc_t400, df_cc_t200, how = 'inner', on = ['SN_KEY', 'MID'])

In [33]:
### 진료날짜순으로 정렬
df_cs_cc = df_cs_cc.sort_values(by = ['SN_KEY','RECU_FR_DD'], ascending = True) # 건강검진 (문진, 결과 및 판정)

In [35]:
df_cs_cc.to_csv('./PP_csv/cs_cc.csv', index = False)

[완료]
- t200, t400 합치기
- 병기조사, 건강검진, 암문진, 진료테이블 합치는 작업은 재발시기 구한 뒤

-----------------------------------------------